Multi-stage inventory problem

Victor M. Zavala 2016 

University of Wisconsin-Madison  

In [123]:
using JuMP 
using Ipopt 

In [124]:
# Model parameters
NST =3;
STAGE = collect(1:NST);
SCEN0 = collect(1:1);
SCENK = collect(1:2);

Demd = Dict()

# construct scenario tree
Dem=zeros(length(STAGE),length(SCEN0),length(SCENK),length(SCENK))
  p=zeros(length(SCEN0),length(SCENK),length(SCENK))
for st in STAGE, i in SCEN0, j in SCENK, k in SCENK
    p[i,j,k]=1/4;
    if st == 1 && i == 1
    Dem[st,i,j,k]=0.0
    end
    if st == 2 && i == 1 && j == 1
    Dem[st,i,j,k]=5.0
    end
    if st == 2 && i == 1 && j == 2
    Dem[st,i,j,k]=25.0
    end
    if st == 3 && i == 1 && j == 1 && k == 1
    Dem[st,i,j,k]=0.0
    end
    if st == 3 && i == 1 && j == 1 && k == 2
    Dem[st,i,j,k]=25.0
    end
    if st == 3 && i == 1 && j == 2 && k == 1
    Dem[st,i,j,k]=50.0
    end   
    if st == 3 && i == 1 && j == 2 && k == 2
    Dem[st,i,j,k]=25.0
    end       
end

LoadError: LoadError: TypeError: Dict: in parameter, expected Type{T}, got Array{Int64,1}
while loading In[124], in expression starting on line 7

In [125]:
# Create Model (standard)
m = Model(solver=IpoptSolver(print_level=0))

@variable(m, 0<=Inv[STAGE,SCEN0,SCENK,SCENK] <=1000)    
@variable(m, 0<=Buy[STAGE,SCEN0,SCENK,SCENK] <=25)

@objective(m, Min,  sum{p[i,j,k]*Buy[st,i,j,k], st in STAGE, i in SCEN0, j in SCENK, k in SCENK}
                  + sum{ p[i,j,k]*Inv[1,i,j,k],              i in SCEN0, j in SCENK, k in SCENK})

@constraint(m, consinv[st in 1:NST-1,i in SCEN0,j in SCENK,k in SCENK], 
               Inv[st+1,i,j,k] == Inv[st,i,j,k] + Buy[st,i,j,k] - Dem[st,i,j,k])

# non-anticipativity stage 1
@constraint(m,  nonantInv1[st in STAGE,i in SCEN0,j in SCENK,k in SCENK; st==1 && i==1],Inv[st,i,j,k] == Inv[st,1,1,1])
@constraint(m,  nonantBuy1[st in STAGE,i in SCEN0,j in SCENK,k in SCENK; st==1 && i==1],Buy[st,i,j,k] == Buy[st,1,1,1])

# non-anticipativity stage 2
@constraint(m, nonantBuy21[st in STAGE,i in SCEN0,j in SCENK,k in SCENK; st==2 && i==1 && j==1],Buy[st,i,j,k] == Buy[st,1,1,1])
@constraint(m, nonantBuy22[st in STAGE,i in SCEN0,j in SCENK,k in SCENK; st==2 && i==1 && j==2],Buy[st,i,j,k] == Buy[st,1,2,1])

# terminal constraint
@constraint(m,    finalInv[ i in SCEN0,j in SCENK,k in SCENK],Inv[NST,i,j,k]==100)

solve(m)

:Optimal

In [126]:
# Results
println(getvalue(Buy))
println(getvalue(Inv))
println("obj ",getobjectivevalue(m))

Buy: 4 dimensions:
[1,:,:,:]
  [1,1,:,:]
    [1,1,1,:]
      [1,1,1,1] = 11.166105824213341
      [1,1,1,2] = 11.166105824176794
    [1,1,2,:]
      [1,1,2,1] = 11.166105824260331
      [1,1,2,2] = 11.166105824260331
[2,:,:,:]
  [2,1,:,:]
    [2,1,1,:]
      [2,1,1,1] = 2.956457913735106
      [2,1,1,2] = 2.956457913745548
    [2,1,2,:]
      [2,1,2,1] = 22.95645791332787
      [2,1,2,2] = 22.95645791332787
[3,:,:,:]
  [3,1,:,:]
    [3,1,1,:]
      [3,1,1,1] = 0.38831141321459295
      [3,1,1,2] = 0.38831141321459295
    [3,1,2,:]
      [3,1,2,1] = 0.38831141321459295
      [3,1,2,2] = 0.38831141321459295

Inv: 4 dimensions:
[1,:,:,:]
  [1,1,:,:]
    [1,1,1,:]
      [1,1,1,1] = 90.8774362622235
      [1,1,1,2] = 90.87743626218696
    [1,1,2,:]
      [1,1,2,1] = 90.87743626227049
      [1,1,2,2] = 90.87743626227049
[2,:,:,:]
  [2,1,:,:]
    [2,1,1,:]
      [2,1,1,1] = 102.0435420863795
      [2,1,1,2] = 102.04354208632729
    [2,1,2,:]
      [2,1,2,1] = 102.0435420865779
      [2,1,2,2]